In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

from cfd_model.cfd_model import load_model, get_generated_dataset
from cfd_model.cfd_dataset import get_dataset
from project_constants import DATASET_FILE, MODEL_FOLDER, WINDOW, DATASET_GENERATED_FILE
from image_functions import preprocess_frame, plot_compare_frames, plot_frame
from plot_functions import plot_frame_histogram

MODEL_FILE = os.path.join(MODEL_FOLDER, "model_20240511_1704.h5")

In [ ]:
dataset, X_train, y_train, X_val, y_val = get_dataset(DATASET_FILE)

# X_train.shape

In [ ]:
model = load_model(MODEL_FILE)
# model.summary()

In [ ]:
training_evaluation = model.evaluate(X_train, [X_train, y_train])
testing_evaluation = model.evaluate(X_val, [X_val, y_val])

In [ ]:
# Visualize Encoder-Decoder
sequence_id = 1900
example_sequence = X_train[sequence_id].copy()
# print("example_sequence:", np.min(example_sequence), np.max(example_sequence), np.mean(example_sequence), np.median(example_sequence), np.var(example_sequence))

reconstructed_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
# print("reconstructed_sequence:", np.min(reconstructed_sequence), np.max(reconstructed_sequence), np.mean(reconstructed_sequence), np.median(reconstructed_sequence), np.var(reconstructed_sequence))

for i in range(len(example_sequence)):
    original = example_sequence[i]
    preprocess_frame(original)
    reconstructed = reconstructed_sequence[i]
    preprocess_frame(reconstructed)
    plot_compare_frames(original, reconstructed, 
                        f"Sequence {sequence_id} (Frame {i}): Original", f"Sequence {sequence_id} (Frame{i}): Reconstructed",
                        sequence_id, i, 
                        orientation="v")
    plt.show()


In [ ]:
# Visualize Encoded
example_sequence = X_train[100].copy()
example_frame = example_sequence[0].copy()

preprocess_frame(example_frame)
plot_frame(example_frame, title="Original Frame")

# Visualize Reconstructed
example_sequence = X_train[100].copy()

reconstructed_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
reconstructed_frame = reconstructed_sequence[0]

preprocess_frame(reconstructed_frame)
plot_frame(reconstructed_frame, title="Reconstructed Frame")

In [ ]:
# Reconstructed Histogram
sequence_id = 1900
frame_id = 0
example_sequence = X_train[sequence_id].copy()
example_frame = example_sequence[0].copy()
reconstructed_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
reconstructed_frame = reconstructed_sequence[frame_id]

preprocess_frame(example_frame)
preprocess_frame(reconstructed_frame)

plot_frame_histogram(example_frame, f"Sequence {sequence_id} (Frame {frame_id}): Original")

plot_frame_histogram(reconstructed_frame, f"Sequence {sequence_id} (Frame {frame_id}): Reconstructed")


In [ ]:
# Compare histograms
example_sequence = X_train[100].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0]
predicted_frame = predicted_sequence[0][0]
frame_min=np.min(frame[frame!=-1])
frame_max=np.max(frame)

frame_data = frame.flatten()
predicted_data = predicted_frame.flatten()

frame_data = np.delete(frame_data, np.where(frame_data == -1))

predicted_data = np.delete(predicted_data, np.where(predicted_data<frame_min))
predicted_data = np.delete(predicted_data, np.where(predicted_data>frame_max))


hist_frame, bin_edges_frame = np.histogram(frame_data, bins='auto')
bin_center_frame = (bin_edges_frame[:-1] + bin_edges_frame[1:])/2
hist_predicted, bin_edges_predicted = np.histogram(predicted_data, bins='auto')
bin_center_predicted = (bin_edges_predicted[:-1] + bin_edges_predicted[1:])/2

plt.figure()
plt.figure(figsize=(20,6))
plt.errorbar(bin_center_frame, hist_frame, yerr=0, fmt='.', color="green")
plt.errorbar(bin_center_predicted, hist_predicted, yerr=0, fmt='|', color="blue")
plt.show()

In [ ]:
# Visualize residuals
sequence_id = 1900
frame_id = 0

example_sequence = X_train[sequence_id].copy()
frame = example_sequence[0].copy()
predicted_sequence = model.predict(np.expand_dims(example_sequence, axis=0), verbose=0)[0][0]
predicted_frame = predicted_sequence[frame_id]

# residual = np.abs(frame - predicted_frame)
residual = np.square(frame - predicted_frame)

plt.clf()
plt.imshow(np.squeeze(residual).transpose(), cmap="Grays")
plt.colorbar(label="error", orientation="horizontal")

In [ ]:
# Plot original vs generated frames

sequence_id = 19
example_sequence = dataset[sequence_id].copy()
generated_sequence = get_generated_dataset(DATASET_GENERATED_FILE)[sequence_id].copy()
# print(example_sequence.shape)
# print(generated_sequence.shape)


begining = 300
limit = 5
end = begining + limit
example_sequence = example_sequence[begining:end]
generated_sequence = generated_sequence[begining:end]

# print(example_sequence.shape)

for i in range(limit):
    original = example_sequence[i]
    preprocess_frame(original)
    generated = generated_sequence[i]
    preprocess_frame(generated)
    plot_compare_frames(original, generated, 
                        f"Sequence {sequence_id} (Frame {i+begining}): Original", f"Sequence {sequence_id} (Frame {i+begining}): Generated",
                        sequence_id, i, 
                        orientation="v")
    plt.show()

In [ ]:
# Generator Histogram
sequence_id = 25
frame_id = 300
example_sequence = dataset[sequence_id].copy()
generated_sequence = get_generated_dataset(DATASET_GENERATED_FILE)[sequence_id].copy()
example_frame = example_sequence[frame_id]
generated_frame = generated_sequence[frame_id]
preprocess_frame(example_frame)
preprocess_frame(generated_frame)

plot_frame_histogram(example_frame, f"Sequence {sequence_id} (Frame {frame_id}): Original")

plot_frame_histogram(generated_frame, f"Sequence {sequence_id} (Frame {frame_id}): Generated")

In [ ]:
dataset, _, _, _, _ = get_dataset(DATASET_FILE, split=False)
generated_dataset = get_generated_dataset(DATASET_GENERATED_FILE)

print(dataset.shape)
print(generated_dataset.shape)


In [ ]:
# frame_id = 300

# original_frame = original_sequence[frame_id]
# generated_frame = generated_sequence[frame_id]

# preprocess_frame(original_frame)
# preprocess_frame(generated_frame)

# original_frame_data = original_frame.flatten()
# generated_frame_data = generated_frame.flatten()


# # remove obstacle
# original_frame_data = np.delete(original_frame_data, np.where(np.isnan(original_frame_data)))
# generated_frame_data = np.delete(generated_frame_data, np.where(np.isnan(generated_frame_data)))

# bins = np.histogram(np.hstack((original_frame_data, generated_frame_data)), bins=100)[1]

# F_original, bins_original = np.histogram(original_frame_data, bins=bins)
# F_generated, bins_generated = np.histogram(generated_frame_data, bins=bins)
# delta_F = abs(F_original-F_generated)

# print(F_original)
# print(F_generated)
# print(delta_F)

In [ ]:
sequence_id = 34
original_sequence = dataset[sequence_id].copy()
generated_sequence = generated_dataset[sequence_id].copy()

# print(original_sequence.shape)
# print(generated_sequence.shape)

#############################################################

original_data = original_sequence.flatten()
generated_data = generated_sequence.flatten()

preprocess_frame(original_data)
preprocess_frame(generated_data)

# remove obstacle
original_data = np.delete(original_data, np.where(np.isnan(original_data)))
generated_data = np.delete(generated_data, np.where(np.isnan(generated_data)))

bins = 100
# bins = 'auto'

bin_edges = np.histogram(np.hstack((original_data, generated_data)), bins=bins)[1]

F_original = np.histogram(original_data, bins=bin_edges)[0].astype(float)
F_generated = np.histogram(generated_data, bins=bin_edges)[0].astype(float)

delta_F = abs(F_original-F_generated)

percentage_F = np.divide(delta_F, F_original, out=np.zeros_like(delta_F), where=F_original!=0) * 100

import sys
np.set_printoptions(threshold=sys.maxsize)

# print(F_original)
# print(F_generated)
# print(delta_F)
# print(percentage_F)

In [ ]:
# plt.clf()
# plt.xlabel('Delta')
# plt.ylabel('Frequency')
# plt.hist(delta_F, bins=100)
# plt.show()

plt.clf()

plt.xlabel('Error percentage')
plt.ylabel('Frequency (MSE)')
plt.hist(percentage_F, bins=100)
plt.title("Generator loss")
plt.show()